In [29]:
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
import yaml
import seaborn as sns

import os

In [30]:
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
config_path = os.path.join(project_root, "config.yaml")

with open(config_path, "r") as f:
    config = yaml.safe_load(f)

In [31]:
csv_path = os.path.join(config['paths']['data_dir'], "data", "pokemon.csv")
df = pd.read_csv(csv_path, encoding='UTF-16', delimiter='\t', index_col=0)

In [32]:
df.head()

,gen,english_name,japanese_name,primary_type,secondary_type,classification,percent_male,percent_female,height_m,weight_kg,...,evochain_1,evochain_2,evochain_3,evochain_4,evochain_5,evochain_6,gigantamax,mega_evolution,mega_evolution_alt,description
national_number,,,,,,,,,,,,,,,,,,,,,
1,I,Bulbasaur,Fushigidane,grass,poison,Seed Pokémon,88.14,11.86,0.7,6.9,...,Level,Ivysaur,Level,Venusaur,NaN,NaN,NaN,NaN,NaN,There is a plant seed on its back right from t...
2,I,Ivysaur,Fushigisou,grass,poison,Seed Pokémon,88.14,11.86,1.0,13.0,...,Level,Ivysaur,Level,Venusaur,NaN,NaN,NaN,NaN,NaN,"When the bulb on its back grows large, it appe..."
3,I,Venusaur,Fushigibana,grass,poison,Seed Pokémon,88.14,11.86,2.0,100.0,...,Level,Ivysaur,Level,Venusaur,NaN,NaN,Gigantamax Venusaur,Mega Venusaur,NaN,Its plant blooms when it is absorbing solar en...
4,I,Charmander,Hitokage,fire,NaN,Lizard Pokémon,88.14,11.86,0.6,8.5,...,Level,Charmeleon,Level,Charizard,NaN,NaN,NaN,NaN,NaN,It has a preference for hot things. When it ra...
5,I,Charmeleon,Lizardo,fire,NaN,Flame Pokémon,88.14,11.86,1.1,19.0,...,Level,Charmeleon,Level,Charizard,NaN,NaN,NaN,NaN,NaN,"It has a barbaric nature. In battle, it whips ..."


In [33]:
print(f'Dataset_length: {len(df)}')
print(f'Dataset_shape: {df.shape}')
print(f'Dataset_dtype: {df.dtypes}')
print(f'Dataset_columns: {df.columns}')

Dataset_length: 898
Dataset_shape: (898, 54)
Dataset_dtype: gen                    object
english_name           object
japanese_name          object
primary_type           object
secondary_type         object
classification         object
percent_male           object
percent_female         object
height_m              float64
weight_kg             float64
capture_rate           object
base_egg_steps          int64
hp                      int64
attack                  int64
defense                 int64
sp_attack               int64
sp_defense              int64
speed                   int64
abilities_0            object
abilities_1            object
abilities_2            object
abilities_hidden       object
against_normal        float64
against_fire          float64
against_water         float64
against_electric      float64
against_grass         float64
against_ice           float64
against_fighting      float64
against_poison        float64
against_ground        float64
against_fl

In [34]:
print(df['description'].iloc[0])

There is a plant seed on its back right from the day this Pokémon is born. The seed slowly grows larger.


In [35]:
interesting_columns = [
    'english_name',
    'japanese_name',
    'primary_type',
    'secondary_type',
    'classification',
    'height_m',
    'weight_kg',
    'abilities_0',
    'abilities_1',
    'abilities_2',
    'abilities_hidden',
    'is_legendary',
    'is_mythical',
    'is_sublegendary',
    'mega_evolution',
    'description'
]

df['full_description'] = "This is a pokemon description. "
for col in interesting_columns:
    col_name = col.replace('_', ' ').capitalize()
    df['full_description'] += col_name + ' is ' +df[col].astype(str).replace({'0':'False', '1': 'True'}) + '. ' if col != 'description' else col_name + ' is ' +df[col].astype(str).replace({'0':'False', '1': 'True'})

In [36]:
a = df['full_description'].iloc[0]
print(a, len(a))

This is a pokemon description. English name is Bulbasaur. Japanese name is Fushigidane. Primary type is grass. Secondary type is poison. Classification is Seed Pokémon. Height m is 0.7. Weight kg is 6.9. Abilities 0 is Overgrow. Abilities 1 is nan. Abilities 2 is nan. Abilities hidden is Chlorophyll. Is legendary is False. Is mythical is False. Is sublegendary is False. Mega evolution is nan. Description is There is a plant seed on its back right from the day this Pokémon is born. The seed slowly grows larger. 515


In [37]:
df['full_description'].to_csv('text_description.csv', sep=',', encoding='utf-8', index=False)

In [38]:



tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-mini")
model = BertModel.from_pretrained("prajjwal1/bert-mini")

In [39]:
inputs = tokenizer(df['full_description'].iloc[0],
    return_tensors="pt",
    return_attention_mask=True)
print(len(inputs['input_ids'][0]))
# Forward pass through the model
outputs = model(**inputs)

# Get last hidden states
last_hidden_states = outputs.last_hidden_state

127


In [40]:
last_hidden_states.shape

torch.Size([1, 127, 256])